In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense

In [ ]:
# 開啟萃取圖檔稍微看一下樣子
img = Image.open("test.bmp")
test_data = np.asarray(img)
plt.imshow(test_data,cmap='gray')
img.close()

hack_img = test_data

In [ ]:
# 載入原本的模組
load_model = tf.keras.models.load_model('.\\mnist_basic_model.h5')
load_model.summary()

In [ ]:
# 宣告一個結構一樣的模組，但是最後一層的激勵函數改掉
new_model = Sequential()
new_model.add(Dense(128, input_dim=784, activation=tf.nn.relu))
new_model.add(Dense(10, activation='linear'))
new_model.load_weights('mnist_basic_model.h5')

In [ ]:
#轉換輸入圖片
hack_data = hack_img.reshape(1,784)
hack_data = hack_data.astype('float32')
hack_data  = hack_data / 255.0

num_classes=10
target_label = tf.keras.utils.to_categorical([3], num_classes)

In [ ]:
# 稍微用這個範例感受一下它做的事情
outputs = new_model.predict(hack_data)
print('outputs:',outputs)

attack_target = 9
one_hot_label = (tf.eye(len(target_label[0])))[attack_target]
print("one_hot_label:",one_hot_label)

tmp = (1 - one_hot_label)*outputs 
tmp = tf.reduce_max(tmp)
print("tmp:",tmp)

tmp1 = (one_hot_label)*outputs 
tmp1 = (one_hot_label*outputs)[0][attack_target]
print("tmp1:",tmp1)

print("max:",tf.reduce_max([tmp,tmp1]))

In [ ]:
# 想辦法透過 CW 演算法針對載入的圖檔做出生成式對抗樣本, 看能不能把這張圖變成 0~9

# 定義 f 函式
def f(input_x,attack_target,kappa=0) :    
    outputs = new_model(input_x)
    # tf.eye 產生單位矩陣用的函式
    one_hot_label = (tf.eye(len(target_label[0])))[attack_target]
    i = tf.reduce_max((1 - one_hot_label)*outputs)
    j = (one_hot_label*outputs)[0][attack_target]
    return tf.reduce_max([i-j,-kappa])

In [ ]:
# 假設把它變成 9
attack_target = 9

# 初始化 c,w 參數
c=1e-3
w = tf.Variable(np.zeros(hack_data.shape,dtype=np.float32))

In [ ]:
for i in range(2000) :
    with tf.GradientTape() as tape:  
        a = 1/2*(tf.math.tanh(w) + 1)    
        loss1 = tf.keras.losses.MeanSquaredError(reduction='sum')(a,hack_data)
        loss2 = c*f(a,attack_target)        
        loss = loss1 + loss2
        
    gradients = tape.gradient(loss, w)
    
    # 參考 https://medium.com/@shuangzizuobh2/using-tensorflow-optimizers-to-minimize-a-simple-function-39681c5b6e72
    tf.optimizers.Adam().apply_gradients(zip([gradients],[w]))
    
    if i%50 == 0 :
        res = 1/2*(tf.math.tanh(w) + 1)
        print(i,' : ',np.argmax(new_model.predict(res)))
        print('loss:',loss)
        print('loss1:',loss1,' loss2:',loss2) 
        if np.argmax(new_model.predict(res)) == attack_target :
            break
                

hack_img = res.numpy()
hack_img = hack_img*255
hack_img = hack_img.reshape(28,28)

In [ ]:

hack_img = hack_img.astype(np.uint8)
im = Image.fromarray(hack_img)
im.save("hack.bmp")

test_data = np.asarray(hack_img)
plt.imshow(test_data,cmap='gray')